In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 11.6 MB/s eta 0:00:00


In [ ]:
import time
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

# === 설정 ===
TARGET_URL = "https://m.stock.naver.com/worldstock/stock/WMT/discuss"
# 원하는 게시 날짜 범위 (2024년 9월 1일 ~ 9월 30일)
START_DATE = datetime(2024, 1, 1)
END_DATE   = datetime(2025, 1, 31)

# === Selenium Chrome 드라이버 설정 (헤드리스 모드 및 리소스 차단) ===
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
prefs = {"profile.managed_default_content_settings.images": 2}  # 이미지 로딩 차단
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)

# 대상 페이지 접속 및 초기 로딩 대기
driver.get(TARGET_URL)
time.sleep(1)

# 마지막 출력된 날짜를 저장하여 중복 출력 방지
last_printed_date = None

# === 스크롤 및 "더보기" 버튼 클릭 반복 (최적화) ===
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # 동적 로딩 대기

    info_containers = driver.find_elements(By.CSS_SELECTOR, "div.DiscussListItem_info__YQ8h9")
    if info_containers:
        last_info = info_containers[-1]
        span_tags = last_info.find_elements(By.CSS_SELECTOR, "span.DiscussListItem_desc__cdfNH")
        if len(span_tags) >= 2:
            posted_time_text = span_tags[1].text.strip()
            if re.match(r'^\d{4}\.\d{2}\.\d{2}', posted_time_text):
                date_str = posted_time_text.rstrip('.')  # 끝의 점 제거
                try:
                    last_date = datetime.strptime(date_str, "%Y.%m.%d")
                    # 날짜가 갱신된 경우에만 출력
                    if last_printed_date != last_date:
                        print("현재 마지막 댓글 날짜:", last_date.strftime("%Y.%m.%d"))
                        last_printed_date = last_date
                    if last_date < START_DATE:
                        print(f"마지막 댓글 날짜 {last_date.strftime('%Y.%m.%d')}가 {START_DATE.strftime('%Y.%m.%d')}보다 이전입니다. 로드를 중단합니다.")
                        break
                except Exception:
                    pass
    more_buttons = driver.find_elements(By.CSS_SELECTOR, "button.InfinityMoreButton_button__ETrlQ")

    if more_buttons:
        # 더보기 버튼이 있다면 JavaScript로 클릭
        driver.execute_script("arguments[0].click();", more_buttons[0])
        time.sleep(1)
    else:
        print("더보기 버튼을 찾을 수 없어 반복 종료")
        break

# 페이지 로드 완료 후 HTML 가져오기 및 드라이버 종료
html = driver.page_source
driver.quit()

# === BeautifulSoup을 이용한 데이터 추출 및 필터링 ===
soup = BeautifulSoup(html, "html.parser")
links = soup.find_all("a", class_="DiscussListItem_link__ruKgh")
data_list = []

for link in links:
    text_tag = link.find("p", class_="DiscussListItem_text__OKI6X")
    if not text_tag:
        continue
    comment_text = text_tag.get_text(separator="\n", strip=True)
    info_div = link.find("div", class_="DiscussListItem_info__YQ8h9")
    if info_div:
        span_tags = info_div.find_all("span", class_="DiscussListItem_desc__cdfNH")
        if len(span_tags) >= 2:
            posted_time_text = span_tags[1].get_text(strip=True)
            if re.match(r'^\d{4}\.\d{2}\.\d{2}', posted_time_text):
                date_str = posted_time_text.rstrip('.')
                try:
                    post_date = datetime.strptime(date_str, "%Y.%m.%d")
                    if START_DATE <= post_date <= END_DATE:
                        data_list.append({
                            "date": date_str,
                            "comment": comment_text
                        })
                except Exception:
                    continue

df = pd.DataFrame(data_list)
print(df)
df.to_csv("wmt_1year.csv", index=False)
df.to_excel("WMT_1year.xlsx", index=False)


현재 마지막 댓글 날짜: 2024.09.10
현재 마지막 댓글 날짜: 2024.07.15
현재 마지막 댓글 날짜: 2024.02.26
현재 마지막 댓글 날짜: 2023.10.29
마지막 댓글 날짜 2023.10.29가 2024.01.01보다 이전입니다. 로드를 중단합니다.
           date                                            comment
0    2025.01.29  "양자컴퓨터 불치병 잡는다. 항암 신약물질 찾기 1만배 빨라졌다." 양자컴퓨터를 활...
1    2025.01.29  그야말로 넘사벽(wall)임\n도저히 넘을 수 없는 벽을 지닌 기업이 바로 월마트이...
2    2025.01.28                                           월마트 시즌 2
3    2025.01.25  엔비디아가 "음성인식AI" 기술의 선두주자 "사운드하운드AI"에 전략적 지분 투자를...
4    2025.01.24                                       월마트는 지존인듯...
..          ...                                                ...
104  2024.02.14           이런 종목 이었나요?\n분할이라는 호재에도 움직임 없네요\n아주 조용..
105  2024.02.10  도무지 주가가 빠지지가 않음\n실적이 크게 미스나야 한번씩 폭락할까 말까임\n미국에...
106  2024.02.09        이젠 분기 배당으로 가자!\n국내주식도 힘들고 안전하고 편안한 주식으로 가자!
107  2024.02.03                                                알간?
108  2024.01.26  홍해 해상화물 운송 차질 심화로 글로벌 기업들 항공화물 운송으로 수요 대체, 페덱스...

[109 rows x 2 columns]


In [ ]:
import time
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

# === 설정 ===
# 대상 종목 리스트
symbols = ["GOOG", "AMZN", "META", "AVGO", "TSLA"]
START_DATE = datetime(2024, 1, 1)
END_DATE   = datetime(2025, 1, 31)

# === Selenium Chrome 드라이버 설정 (헤드리스 모드 및 이미지 로딩 차단) ===
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.set_window_size(1920, 1080)

for symbol in symbols:
    # 종목에 대해 URL에 들어갈 코드: 이미 ".O"가 없으면 붙임.
    url_symbol = symbol if symbol.endswith(".O") else symbol + ".O"

    TARGET_URL = f"https://m.stock.naver.com/worldstock/stock/{url_symbol}/discuss"
    print(f"Processing {symbol}: {TARGET_URL}")

    driver.get(TARGET_URL)
    time.sleep(0.6)  # 초기 로딩 대기

    last_printed_date = None

    # === 스크롤 및 "더보기" 버튼 클릭 반복 ===
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.6)  # 동적 로딩 대기

        info_containers = driver.find_elements(By.CSS_SELECTOR, "div.DiscussListItem_info__YQ8h9")
        if info_containers:
            last_info = info_containers[-1]
            span_tags = last_info.find_elements(By.CSS_SELECTOR, "span.DiscussListItem_desc__cdfNH")
            if len(span_tags) >= 2:
                posted_time_text = span_tags[1].text.strip()
                if re.match(r'^\d{4}\.\d{2}\.\d{2}', posted_time_text):
                    date_str = posted_time_text.rstrip('.')
                    try:
                        last_date = datetime.strptime(date_str, "%Y.%m.%d")
                        if last_printed_date != last_date:
                            print(f"{symbol} - 현재 마지막 댓글 날짜:", last_date.strftime("%Y.%m.%d"))
                            last_printed_date = last_date
                        if last_date < START_DATE:
                            print(f"{symbol} - 마지막 댓글 날짜 {last_date.strftime('%Y.%m.%d')}가 시작 날짜({START_DATE.strftime('%Y.%m.%d')})보다 이전입니다. 로드를 중단합니다.")
                            break
                    except Exception:
                        pass

        more_buttons = driver.find_elements(By.CSS_SELECTOR, "button.InfinityMoreButton_button__ETrlQ")
        if more_buttons:
            driver.execute_script("arguments[0].click();", more_buttons[0])
            time.sleep(0.1)
        else:
            print(f"{symbol} - 더보기 버튼을 찾을 수 없어 반복 종료")
            break

    html = driver.page_source

    # === BeautifulSoup을 이용한 데이터 추출 및 필터링 ===
    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a", class_="DiscussListItem_link__ruKgh")
    data_list = []

    for link in links:
        text_tag = link.find("p", class_="DiscussListItem_text__OKI6X")
        if not text_tag:
            continue
        comment_text = text_tag.get_text(separator="\n", strip=True)
        info_div = link.find("div", class_="DiscussListItem_info__YQ8h9")
        if info_div:
            span_tags = info_div.find_all("span", class_="DiscussListItem_desc__cdfNH")
            if len(span_tags) >= 2:
                posted_time_text = span_tags[1].get_text(strip=True)
                if re.match(r'^\d{4}\.\d{2}\.\d{2}', posted_time_text):
                    date_str = posted_time_text.rstrip('.')
                    try:
                        post_date = datetime.strptime(date_str, "%Y.%m.%d")
                        if START_DATE <= post_date <= END_DATE:
                            data_list.append({
                                "date": date_str,
                                "comment": comment_text
                            })
                    except Exception:
                        continue

    df = pd.DataFrame(data_list)
    print(df)
    csv_filename = f"{symbol}_1year.csv"
    xlsx_filename = f"{symbol}_1year.xlsx"
    df.to_csv(csv_filename, index=False)
    df.to_excel(xlsx_filename, index=False)
    print(f"{symbol} 데이터 저장 완료: {csv_filename}, {xlsx_filename}\n")

driver.quit()


Processing GOOG: https://m.stock.naver.com/worldstock/stock/GOOG.O/discuss
GOOG - 현재 마지막 댓글 날짜: 2024.07.08
GOOG - 현재 마지막 댓글 날짜: 2024.04.25
GOOG - 현재 마지막 댓글 날짜: 2024.01.31
GOOG - 현재 마지막 댓글 날짜: 2023.10.25
GOOG - 마지막 댓글 날짜 2023.10.25가 시작 날짜(2024.01.01)보다 이전입니다. 로드를 중단합니다.
          date                                            comment
0   2025.01.27  https://blog.naver.com/hun_ch/223740286132\n낚시...
1   2025.01.24  다시 금요일이네요!\n업데이트 된 한국의 해외주식 보관잔량 순위와 가장 많이 매수/...
2   2025.01.14  #카톡검열 징역3년 이라네 국민고발범의 시초 - https://youtube.com...
3   2025.01.07                                              일단 튀자
4   2025.01.03  이번주 ALPHABET  한국인 순매수 규모\n또 다시 금요일이네요.\n국내 순매수...
..         ...                                                ...
85  2024.01.31                                            떨어지면 추매
86  2024.01.31                                쳐 맞아야지 그래냐 다음에 잘 하지
87  2024.01.30                                                 인증
88  2024.01.21  알파벳(Alphabet Inc.)은 2015년 구글의 구조조정과 함께 설립된 지주회...
89  